# Training

In [1]:
import sys, os
import pandas as pd
import numpy as np
import setting
import pathSetting
import torch
sys.path.append("../../..")

from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

import torch

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/programdrive/anaconda3/envs/clust/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda is available.


In [2]:
# Set model methods i.e., 'LSTM_cf', 'GRU_cf', 'CNN_1D_cf', 'LSTM_FCNs_cf', 'FC_cf' 
model_method = 'LSTM_cf'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

In [3]:
get_ms_list = db_client.measurement_list(bucket_name)
get_ms_list

['classification_actionPattern_testX_cleanLevel0',
 'classification_actionPattern_testy_cleanLevel0',
 'classification_actionPattern_trainX_cleanLevel0',
 'classification_actionPattern_trainy_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel4',
 'forecasting_strawberryOpen_test_cleanLevel0',
 'forecasting_strawberryOpen_test_cleanLevel4',
 'forecasting_strawberryOpen_train_cleanLevel0',
 'forecasting_strawberryOpen_train_cleanLevel4',
 'regression_energy_testX_cleanLevel0',
 'regression_energy_testX_cleanLevel4',
 'regression_energy_testy_cleanLevel0',
 'regres

In [4]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_strawberryOpen',
 'regression_energy',
 'forecasting_Hs2SwineFarmWithWeatherTime',
 'forecasting_gunwiStrawberryWeather',
 'classification_actionPattern']

In [5]:
data_name_X = 'classification_actionPattern_trainX_cleanLevel0'
dataX = db_client.get_data(bucket_name, data_name_X)

# datay
data_name_y = 'classification_actionPattern_trainy_cleanLevel0'
datay = db_client.get_data(bucket_name, data_name_y)

dataset_name = collection_list[4]
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
clean_level = data_meta["clean_level"]
integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

In [6]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)

# 2-2
# cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함
# cleanTrainDataParam = clean_level --> clean_level로 변경

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam='noscale'
scaleMethod='minmax'

# 2-4
splitRatio = 0.8
mode = 'Classification'

# 2-5
scalerRootPath_X = os.path.join(scalerPath, data_name_X, str(clean_level))
scalerRootPath_y = os.path.join(scalerPath, data_name_y, str(clean_level))
train_X, val_X, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod, mode)
train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, "NoScale", scaleMethod, mode)

In [7]:
transformParameter = {
    'window_num':128
}

window_num = 0
dim = 3

if model_method == "FC_cf":
    dim = 2

if type(train_X) !=  np.ndarray:
    train_X, train_y = transDFtoNP(train_X, train_y, window_num, dim)
    val_X, val_y = transDFtoNP(val_X, val_y, window_num, dim)

input_size = train_X.shape[1]
if dim != 2:
    seq_len = train_X.shape[2] # seq_length

/home/jhlee/CLUST_KETI/Clust/examples/7. ModelTest/7-1. Classification/../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
/home/jhlee/CLUST_KETI/Clust/examples/7. ModelTest/7-1. Classification/../../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]
/home/jhlee/CLUST_KETI/Clust/examples/7. ModelTest/7-1. Classification/../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timez

In [8]:
print(type(train_X), train_X.shape)
print(train_X)

<class 'numpy.ndarray'> (5881, 9, 128)
[[[ 1.808515e-04  1.013856e-02  9.275574e-03 ... -1.147484e-03
   -2.222655e-04  1.575500e-03]
  [ 1.076681e-02  6.579480e-03  8.928878e-03 ...  1.714439e-04
    1.574181e-03  3.070189e-03]
  [ 5.556068e-02  5.512483e-02  4.840473e-02 ...  2.647864e-03
    2.381057e-03 -2.269757e-03]
  ...
  [ 1.012817e+00  1.022833e+00  1.022028e+00 ...  1.018445e+00
    1.019372e+00  1.021171e+00]
  [-1.232167e-01 -1.268756e-01 -1.240037e-01 ... -1.240696e-01
   -1.227451e-01 -1.213260e-01]
  [ 1.029341e-01  1.056872e-01  1.021025e-01 ...  1.003852e-01
    9.987355e-02  9.498741e-02]]

 [[ 1.093752e-03  4.550077e-03  2.879173e-03 ... -4.646144e-03
   -2.941333e-03 -1.598751e-03]
  [-4.687588e-03 -7.487894e-03 -8.429991e-03 ... -5.479850e-03
   -2.987481e-03 -7.192009e-03]
  [-2.685954e-02 -2.509841e-02 -2.597534e-02 ... -2.996671e-04
   -4.886324e-03 -3.397195e-03]
  ...
  [ 1.018851e+00  1.022380e+00  1.020781e+00 ...  1.014788e+00
    1.016499e+00  1.017849e+0

In [9]:
# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_cf' or model_method == 'GRU_cf':
    modelParameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'num_classes': 6,
        'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bidirectional': True  # 모델의 양방향성 여부, bool(default: True)   
    }
    if model_method == 'LSTM_cf':
        modelParameter['rnn_type'] = 'lstm'
    else:
        modelParameter['rnn_type'] = 'gru'
        
# CNN_1D model parameters
elif model_method == 'CNN_1D_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
    'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
    'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
    'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
    'drop_out': 0.1 # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
    'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
    'fc_drop_out': 0.1 # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# FC model parameters
elif model_method == 'FC_cf':
    modelParameter = {
    'input_size': input_size,
    'num_classes': 6,
    'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    'bias': True# bias 사용 여부, bool(default: True)
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': device, 
    'n_epochs': 5, 
    'batch_size': 16
}

In [10]:
modelTags =["action", "sensor", "classification", "pattern"]
trainDataType = "timeseries"
modelPurpose = "classification"

# # 2
trainDataInfo = data_meta

# 3. 모델을 저장할 파일 패스를 생성한다.
model_name = None
if model_name is None:
    model_name = dataset_name + '_' + model_method + '_cleanLevel' + str(clean_level)
else:
    pass

print(model_name)

from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, data_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

classification_actionPattern_LSTM_cf_cleanLevel0
./Models/LSTM_cf/classification_actionPattern_LSTM_cf_cleanLevel0/classification_actionPattern_trainX_cleanLevel0/5047520731b0a365ac0d5de8a395a813/model.pkl


In [11]:
from Clust.clust.ML.classification.train import ClassificationTrain as CML

cml = CML()
cml.set_param(trainParameter)
cml.set_model(model_method, modelParameter)
cml.set_data(train_X, train_y, val_X, val_y)
cml.train()
cml.save_best_model(modelFilePath)

Start training model


/programdrive/anaconda3/envs/clust/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")



Epoch 1/5
train Loss: 1.5537 Acc: 0.3351
val Loss: 1.3733 Acc: 0.3950

Training complete in 0m 24s
Best val Acc: 0.617267


In [12]:
model_meta = {
    "modelName": model_name,
    "trainDataInfo": trainDataInfo,
    "featureList": featureListX,
    "target": featureListy,
    "trainDataType": trainDataType,
    "modelPurpose": modelPurpose,
    "model_method": model_method,
    "modelTags": modelTags,
    "trainDataName": [data_name_X,data_name_y],
    "cleanLevel":clean_level,
    "NaNProcessingParam":NaNProcessingParam,
    "scalerParam": scalerParam,
    "trainParameter": trainParameter,
    "modelParameter": modelParameter,
    
    "files":{
            "modelFile":{
                    "fileName":"model.pth",
                    "filePath":modelFilePath
                },
            "XScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":X_scalerFilePath       
                },
            "yScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":y_scalerFilePath       
                }
        }
}

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, model_meta)

========== Data Save Success ==========
{'modelName': 'classification_actionPattern_LSTM_cf_cleanLevel0', 'trainDataInfo': {'data_name': 'classification_actionPattern_trainX_cleanLevel0', 'ingestion_param': {'start_time': '1992-1-11', 'end_time': '2012-02-27', 'ms_list_info': [['bio_action_sensors', 'accelerationTrain']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 675}, 'clean_level': 0, 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False}}, 'imputation_param': {'flag': False}}}, 'featureList': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8'], 'target': ['value'], 'trainDataType': 'timeseries', 'modelPurpose': 'classification', 'model_method': 'LSTM_cf', 'modelTags': ['action', 'sensor', 'classification', 'pattern'],